# Retrieval Augmented Generation(RAG) с использованием GigaChat на примере задачи "разговор с книгой"

Подход RAG позволяет большим языковым моделям (LLM) отвечать на вопросы по документам, которы не помещаются в промпт.
Ниже приведен пример того, как можно научить модель отвечать на вопросы, используя текст из книги "Мастер и Маргарита".

Подробнее про RAG вы можете прочитать в [документации LangChain](https://python.langchain.com/docs/use_cases/question_answering/) и в курсе по промпт-инженирингу от Сбера (ссылка будет позже).

В качестве примера мы рассмотрим текст романа Булгакова "Мастер и Маргарита" (главы 1 и 2).

Вопрос будет - `Какой плащ был у Понтия Пилата?`. Ответ содержится во второй главе книги:
`В белом плаще с кровавым подбоем, шаркающей кавалерийской походкой, ранним утром четырнадцатого числа весеннего месяца нисана в крытую колоннаду между двумя крыльями дворца ирода великого вышел прокуратор Иудеи Понтий Пилат.`

## Установка

Для работы нам понадобится векторая база данных. Мы будем использовать Chroma.

In [4]:
%pip install chromadb


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Инициализация модели
Теперь инициализируем модель GigaChat.

In [5]:
from langchain.chat_models.gigachat import GigaChat

llm = GigaChat(credentials='Yjg4MTQzMmUtNDAwMS00NDk0LThjOGUtNmU5ZWQ2YzQ4NDQ2OmQ4MWMxZGZiLTFmNGYtNDk5NS05OGQzLTBiMzYyYWJmNjk3OA==')

Для проверки спросим у модели вопрос про цвет плаща без какого-либо контекста. Возможно, она и так будет давать ожидаемый ответ...

In [6]:
from langchain.schema import HumanMessage

question = "Какой плащ был у Понтия Пилата?"
llm([HumanMessage(content=question)]).content[0:200]

/home/evgeny/Code/DeepHack/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


ResponseError: (URL('https://ngw.devices.sberbank.ru:9443/api/v2/oauth'), 400, b'{"code":7,"message":"scope from db not fully includes consumed scope"}', Headers([('server', 'nginx'), ('date', 'Thu, 25 Apr 2024 14:16:57 GMT'), ('content-type', 'application/json'), ('content-length', '70'), ('connection', 'keep-alive'), ('vary', 'Origin'), ('vary', 'Access-Control-Request-Method'), ('vary', 'Access-Control-Request-Headers'), ('cache-control', 'no-cache, no-store, max-age=0, must-revalidate'), ('pragma', 'no-cache'), ('expires', '0'), ('x-content-type-options', 'nosniff'), ('strict-transport-security', 'max-age=31536000 ; includeSubDomains'), ('x-frame-options', 'DENY'), ('x-xss-protection', '1 ; mode=block'), ('referrer-policy', 'no-referrer'), ('allow', 'POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains')]))

Видим, что модель не отвечает так, как нам хотелось бы, поэтому применим RAG-подход.

## Подготовка документа

Для работы с документом нам нужно разделить его на части. Для этого используем `TextLoader` для загрузки книги и `RecursiveCharacterTextSplitter`, чтобы разделить текст на приблизительно равные куски в ~1000 символов с перекрытием в ~200 символов. Этот тип сплиттера сам выбирает каким способом следует оптимально разделять документ (по абзацам, по предложениям и т.д.)

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = TextLoader("../test.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 6


После нарезки мы получили 91 документ частями книги.

## Создание базы данных эмбеддингов

Эмбеддинг это векторное представление текста, которое может быть использовано для определения смысловой близости текстов. Векторная база данных хранит тексты и соответствующие им эмбеддинги, а также умеет выполнять поиск по ним. Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

> Сервис эмбеддингов на данный момент находится на этапе предварительного тестирования и может быть не доступен всем пользователям или не стабилен.

> Параметры `one_by_one_mode` и `_debug_delay` используются для работы в режиме отладки и снижения нагрузки на сервер работы с эммбеддингами.

In [2]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(credentials='Yjg4MTQzMmUtNDAwMS00NDk0LThjOGUtNmU5ZWQ2YzQ4NDQ2OmQ4MWMxZGZiLTFmNGYtNDk5NS05OGQzLTBiMzYyYWJmNjk3OA==', scope='GIGACHAT_API_CORP', verify_ssl_certs=False)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

## Поиск по базе данных

Теперь можно обратиться к базе данных и попросить найти документы, которые с наибольшей вероятностью содержат ответ на наш вопрос.

По-умолчанию база данных возвращает 4 наиболее релевантных документа. Этот параметр можно изменить в зависимости от решаемой задачи и типа документов.

Видно, что первый же документ содержит внутри себя часть книги с ответом на наш вопрос.

In [10]:

question = ""
docs = db.similarity_search(question, k=4)

len(docs)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


2

In [11]:
docs

[Document(page_content='a set of embarrassingly simple one-layer linear models named LTSF-Linear for comparison. \nExperimental results on nine real-life datasets show that LTSF-Linear surprisingly \noutperforms existing sophisticated Transformer-based LTSF models in all cases, and \noften by a large margin. Moreover, we conduct comprehensive empirical studies to \nexplore the impacts of various design elements of LTSF models on their temporal relation \nextraction capability. We hope this surprising finding opens up new research directions \nfor the LTSF task. We also advocate revisiting the validity of Transformer-based solutions \nfor other time series analysis tasks (e.g., anomaly detection) in the future. Code is available at: \\url{<a href="https://github.com/cure-lab/LTSF-Linear" \nrel="external noopener nofollow" class="link-external link-https">this https URL</a>}.', metadata={'source': '../test.txt'}),
 Document(page_content='Recently, there has been a surge of Transformer-ba

In [12]:
print(f"... {str(docs[0])[620:800]} ...")

... solutions \nfor other time series analysis tasks (e.g., anomaly detection) in the future. Code is available at: \\url{<a href="https://github.com/cure-lab/LTSF-Linear" \nrel="exter ...


## QnA цепочка

Теперь мы создадим цепочку QnA, которая специально предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель и ретривер (база данных).

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

NameError: name 'llm' is not defined

Наконец можно задать вопрос нашей цепочке и получить правильный ответ!

In [ ]:
qa_chain({"query": question})

{'query': 'Какой плащ носил Понтий Пилат?',
 'result': 'Понтий Пилат носил белый плащ с кровавым подбоем.'}

Несколько дополнительных вопросов для проверки работоспособности:

In [ ]:
qa_chain({"query": "Какая трость была у Воланда?"})

{'query': 'Какая трость была у Воланда?',
 'result': 'Трость, которую держал Воланд, имела черный набалдашник в виде головы пуделя.'}

In [ ]:
qa_chain({"query": "В чем главная проблема человека?"})

{'query': 'В чем главная проблема человека?',
 'result': 'Главная проблема человека, согласно данному контексту, заключается в том, что он не может управлять своей жизнью и всем распорядком на земле без точного плана на некоторый срок.'}